# Interpolation GeoPlots

> These plots are created to plot the results of interpolation over a space for a perticular timestamp. You need to provide shape files and a data frame containing the pollutant levels. The function internally uses `polire`, a library built for interpolation to interpolate the values for the whole region and then forms a contour map of the results.

Standard libraries to be imported for usage

In [1]:
#! /usr/bin/env python
from geopandas import GeoDataFrame
import pandas as pd
import matplotlib.pyplot as plt
from vayu import interpolPlot

In [2]:
"""data files
shape files:
https://www2.census.gov/geo/tiger/TIGER2017/STATE/

data files:
https://aqs.epa.gov/aqsweb/airdata/daily_44201_2019.zip
https://openaq.org/#/countries/IN?_k=j7c7ka
"""
shapeFile = "../data/shapeFiles/tl_2017_us_state.shp"
datafile = "../data/shapeFiles/daily_44201_2018.csv"
long = "Longitude"
lat = "Latitude"
pollutant = "Arithmetic Mean"
df = pd.read_csv(datafile)
shape_df = GeoDataFrame.from_file(shapeFile)
shape_df.drop(shape_df.index[[34, 35, 36, 40, 41, 49, 31]], 
              inplace=True)

FileNotFoundError: [Errno 2] File b'../data/shapeFiles/daily_44201_2018.csv' does not exist: b'../data/shapeFiles/daily_44201_2018.csv'

In [ ]:
interpolPlot(df, shape_df, long, lat, pollutant)
plt.show()

Focusing on Califonria.

In [ ]:
shape_df = shape_df[13:14]

We don't support `LinearSegmentedColormap` pass a `ListedColormap`. We will be demoing below how to convert from former to latter.

In [ ]:
## This will not work

# interpolPlot(df, shape_df, long, lat, pollutant, cmap='Greens')
# plt.show()

In [ ]:
# Converting to ListedColormap

import numpy as np
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

N = 258
cmap_colors = cm.get_cmap('Greens', N)
newcolors = (cmap_colors(np.linspace(0, 1, N))[:, :3]).tolist()
listedcmap = ListedColormap(newcolors, name='GreensListed', N=N)

In [ ]:
interpolPlot(df, shape_df, long, lat, pollutant, cmap=listedcmap)
plt.savefig("interpolPlot2.png")
plt.show()

### Delhi
Let's have a look at pollution data in delhi.

In [ ]:
!ls "../data/shapeFiles/delhi"

In [ ]:
shapeFile = "../data/shapeFiles/delhi/wards delimited.shp"
shape_df = GeoDataFrame.from_file(shapeFile)
datafile = "../data/delhi_1_5_Aug.csv"
df = pd.read_csv(datafile)
df.head()

In [ ]:
# some_value = '2019-07-31'
# df = df.loc[df['local'].str.contains(some_value, regex=False)] # selecting one date

In [ ]:
locations = pd.unique(df.location)
dff = []
for location in locations:
    mask = df['location'] == location
    mean = df[mask].value.mean()
    latitude = df[mask].latitude.mean()
    longitude = df[mask].longitude.mean()
    dff.append([latitude, longitude, mean])

In [ ]:
df = pd.DataFrame(dff, columns=[lat, long, pollutant])
df.head()

In [ ]:
interpolPlot(df, shape_df, long, lat, pollutant, 
             cmap=listedcmap, plot_train_points=True,
             markersize=30)
# plt.savefig("interpolPlot3.png")
plt.show()

In [ ]:
import json

In [ ]:
!curl https://api.openaq.org/v1/measurements?city=Delhi > ../re/test.json
with open('../re/test.json', 'r') as f:
    data = json.load(f)

In [ ]:
df = pd.DataFrame(data['results'])
df.head()